<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-исходных-данных-к-обучению" data-toc-modified-id="Подготовка-исходных-данных-к-обучению-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка исходных данных к обучению</a></span><ul class="toc-item"><li><span><a href="#Загрузка-исходных-данных-и-ознакомление-с-ними" data-toc-modified-id="Загрузка-исходных-данных-и-ознакомление-с-ними-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка исходных данных и ознакомление с ними</a></span></li><li><span><a href="#Очистка-текста-от-лишних-символов-и-его-лемматизация" data-toc-modified-id="Очистка-текста-от-лишних-символов-и-его-лемматизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Очистка текста от лишних символов и его лемматизация</a></span></li><li><span><a href="#Разбиение-датасета-на-обучающую-и-тестовую-выборки" data-toc-modified-id="Разбиение-датасета-на-обучающую-и-тестовую-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Разбиение датасета на обучающую и тестовую выборки</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Обучение-и-тестирование-моделей" data-toc-modified-id="Обучение-и-тестирование-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и тестирование моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-LogisticRegression" data-toc-modified-id="Модель-LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель LogisticRegression</a></span></li><li><span><a href="#Модель-LGBMClassifier" data-toc-modified-id="Модель-LGBMClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель LGBMClassifier</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Классификация комментариев на токсичные и нетоксичные

**Цель проекта:** создать модель, классифицирующую комментарии на токсичные и нетоксичные. Метрика качества модели *F1* должна быть не менее 0,75. Модель планируется использовать в сервисе интернет-магазина для поиска токсичных комментариев и отправки их на модерацию.

**Исходные данные:** набор комментариев с разметкой о токсичности.

**Описание данных:**
Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка исходных данных к обучению

### Загрузка исходных данных и ознакомление с ними

Загружаем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import spacy
import en_core_web_sm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier

Для лучшей визуализации увеличим максимальную отображаемую ширину колонки датафрейма и загрузим исходный файл. 

In [2]:
pd.options.display.max_colwidth = 800
data = pd.read_csv("/datasets/toxic_comments.csv")

Выведем общую информацию об исходных данных.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


В столбце "text" имеется большое количество лишних символов, от которых необходимо будет избавиться.

Изучим столбец "toxic" подробнее, а именно величины, записанные в нем и их баланс.

In [5]:
data["toxic"].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [6]:
(data["toxic"].sum() / data.shape[0] * 100).round(1)

10.2

В столбце "toxic" присутствуют единицы и нули. Всего два класса - здесь ошибки в исходных данных нет. При этом имеется существенный дисбаланс классов. Положительных ответов всего 10% от всего датафрейма. Это необходимо учесть при обучении моделей.

### Очистка текста от лишних символов и его лемматизация

Напишем функции для очистки текста от лишних символов и его лемматизации. При этом учтем, что в английском большое значение имеет символ апострофа ('). Данный символ используется во многих очень распространенных словосочетаниях и словах, например: weren't, it's, don't, boy's и т.д. Таким образом, его замена на пробелы и дальнейшее удаление может затруднить расшифровку слов для алгоритмов, анализирующих текст. Решить данную проблему можно различными методами. Здесь применим следующий подход. Выполним первичную очистку - для нее напишем функцию **clean_first**, в которой символ апострофа включен в шаблон (апостроф не заменяется на пробел). Затем выполним лемматизацию текста с удалением стоп-слов при помощи функции **lemmatize**. При выполнении лемматизации рассматриваемые апострофы уйдут в результате приведения слов к начальной форме, а также удаления стоп-слов. Оставшиеся апострофы являются лишними символами: для их удаления запустим функцию **clean_second**, в которой символ апострофа не включен в шаблон (апостроф заменяется на пробел, лишние пробелы удаляются при помощи комбинации методов split/join).

In [7]:
def clean_first(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z']", " ", text)
    return text

In [8]:
def clean_second(text):
    text = re.sub(r"[^a-zA-Z]", " ", text)
    return " ".join(text.split())

In [9]:
nlp = spacy.load("en_core_web_sm", disable = ['tagger','perser','ner'])

In [10]:
def lemmatize(text):
    temp_str = ""
    for token in nlp(text):
        if token.is_stop == False:
            temp_str += " "
            temp_str += token.lemma_
    return " ".join(temp_str.split())

Запустим первую функцию - **clean_first** и проконтролируем визуально результат, выведя первые пять строк датафрейма.

In [11]:
data["clean_text"] = data["text"].apply(clean_first)
data.head()

,text,toxic,clean_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalisms just closure on some gas after i voted at new york dolls fac and please don't remove the template from the talk page since i'm retired now
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,d'aww he matches this background colour i'm seemingly stuck with thanks talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,hey man i'm really not trying to edit war it's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,more i can't make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it's listed in the relevant form eg wikipedia good article nominations transport
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir are my hero any chance you remember what page that's on


Запустим втору функцию - **lemmatize** и проконтролируем визуально результат. Учитывая длительность выполнения функции установим счетчик прогресса решения задачи (tqdm).

In [12]:
tqdm.pandas()
data["lemm_text"] = data["clean_text"].progress_apply(lemmatize)

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Запустим первую функцию - **clean_second** и проконтролируем визуально результат, выведя первые пять строк датафрейма.

In [13]:
data["lemm_text"] = data["lemm_text"].apply(clean_second)
data.head()

,text,toxic,clean_text,lemm_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalisms just closure on some gas after i voted at new york dolls fac and please don't remove the template from the talk page since i'm retired now,explanation edit username hardcore metallica fan revert vandalisms closure gas vote new york doll fac remove template talk page retire
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,d'aww he matches this background colour i'm seemingly stuck with thanks talk january utc,d aww match background colour seemingly stick thank talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,hey man i'm really not trying to edit war it's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,hey man try edit war guy constantly remove relevant information talk edit instead talk page care format actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,more i can't make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it's listed in the relevant form eg wikipedia good article nominations transport,real suggestion improvement wonder section statistic late subsection type accident think reference need tidy exact format ie date format etc late preference format style reference want let know appear backlog article review guess delay reviewer turn list relevant form eg wikipedia good article nomination transport
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir are my hero any chance you remember what page that's on,sir hero chance remember page


Следует отметить, что выполненная обработка текста не является абсолютно чистой и законченной. Например, не удалены такие словосочетания как https. Наверняка имеются и другие погрешности. Для полной очистки текста, вероятно, требуется серьезная аналитическая работа. В первом приближении примем, что выполненной очистки достаточно для достижения требуемого значения показателя F-1. При необходимости можно будет вернуться к тексту и подготовить его к обучению более тщательно.

### Разбиение датасета на обучающую и тестовую выборки

In [14]:
X = data["lemm_text"]
y = data["toxic"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=56789, test_size=0.25)

In [15]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

### Выводы

1. Исходные данные загружены и изучены. В исходных данных выявлен существенный дисбаланс классов (положительные ответы составляют только 10% датасета).
2. Выполнена очистка текста от лишних символов и его лемматизация.
3. Выполнено разбиение датасета на обучающую и тестовую выборки.

## Обучение и тестирование моделей

Рассмотрим две модели: LogisticRegression и LGBMClassifier (простую и сложную).

### Модель LogisticRegression

Подберем оптимальные гиперпараметры для рассматриваемой модели.

In [16]:
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 1))),
    ('model', LogisticRegression(solver='liblinear', random_state=56789, class_weight="balanced"))
])

params = {
    'model__max_iter': [20, 50, 100],
    "model__C": [0.1, 1.0, 10]
}

cv = KFold(n_splits=3, shuffle=True, random_state=56789)
search_LogReg = GridSearchCV(pipe, param_grid=params, scoring="f1", cv=cv, n_jobs=-1)

In [17]:
%%time
search_LogReg.fit(X_train, y_train)

CPU times: user 6min 4s, sys: 3min 7s, total: 9min 11s
Wall time: 9min 14s


GridSearchCV(cv=KFold(n_splits=3, random_state=56789, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                 

Выведем лучший набор гиперпараметров для модели и соответствующее ему значение F1 метрики.

In [18]:
print(search_LogReg.best_params_)
print(search_LogReg.best_score_)

{'model__C': 10, 'model__max_iter': 20}
0.759960283659724


Получим значение F1 метрики для тестовой выборки при лучшем наборе гиперпараметров.

In [19]:
predictions = search_LogReg.best_estimator_.predict(X_test)
score = f1_score(y_test, predictions)
score

0.7675096142640718

Полученное на тестовой выборке значение F1 метрики удовлетворяет заданному критерию (F1 > 0.75).

### Модель LGBMClassifier

Подберем оптимальные гиперпараметры для рассматриваемой модели.

In [20]:
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 1))),
    ('model', LGBMClassifier(seed=56789, class_weight="balanced"))
])

params = {
    "model__n_estimators": [20, 60],
    "model__learning_rate": [0.5, 0.1]
}

search_LGBMClass = GridSearchCV(pipe, param_grid=params, scoring="f1", cv=cv, n_jobs=-1)

In [21]:
%%time
search_LGBMClass.fit(X_train, y_train)

CPU times: user 12min 53s, sys: 2.47 s, total: 12min 55s
Wall time: 13min 6s


GridSearchCV(cv=KFold(n_splits=3, random_state=56789, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                 

Выведем лучший набор гиперпараметров для модели и соответствующее ему значение F1 метрики.

In [23]:
print(search_LGBMClass.best_params_)
print(search_LGBMClass.best_score_)

{'model__learning_rate': 0.5, 'model__n_estimators': 60}
0.734495023151957


Получим значение F1 метрики для тестовой выборки при лучшем наборе гиперпараметров.

In [24]:
predictions = search_LGBMClass.best_estimator_.predict(X_test)
score = f1_score(y_test, predictions)
score

0.7353982300884956

Полученное на тестовой выборке значение F1 метрики неудовлетворяет заданному критерию (F1 > 0.75).

### Выводы

1. Рассмотрено две модели: LogisticRegression и LGBMClassifier. Для данных моделей подобраны оптимальные гиперпараметры.
2. На тестовой выборке F1 метрика равняется 0,77 и 0,74 для моделей LogisticRegression и LGBMClassifier соответственно. Таким образом, для модели LogisticRegression заданный критерий (F1 > 0.75) достигнут, а для модели LGBMClassifier - нет. Дальнейший подбор гиперпараметров для модели LGBMClassifier с целью повышения F1 метрики не выполнялся ввиду ресурсоемкости данной модели и достижения требуемого критерия моделью LogisticRegression.
3. С учетом быстроты работы и достигнутых значений F1 метрики модель LogisticRegression является оптимальной для решения поставленной в проекте задачи.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны